<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #5: Add new frames to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 8th, 2021</h5>

# Set up and requirements

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.tutorials_utils as t_utils
import utils.t5_utils as t5
import utils.zooniverse_utils as zoo

print("Packages loaded successfully")

### Choose your project

In [ ]:
project = t_utils.choose_project()

### Initiate SQL database and populate sites, movies and species

In [ ]:
# Specify the path of the movies 
movies_path = "/uploads"

# Specify the path of the sql database
db_path = "koster_lab.db"

# Initiate the SQL database 
%run -i "../db_starter/starter.py" --movies_path $movies_path --db_path $db_path --project_name $project.value

# Retrieve classified videos

### Retrieve Zooniverse information

In [ ]:
# Save your Zooniverse user name and password.
zoo_user, zoo_pass = zoo.zoo_credentials()

In [ ]:
# Specify the Zooniverse information required throughout the tutorial
zoo_info = ["subjects", "workflows", "classifications"]

# Retrieve and store the Zooniverse information required throughout the tutorial in a dictionary
zoo_info_dict = zoo.retrieve_zoo_info(zoo_user, zoo_pass, project.value, zoo_info)

In [ ]:
# Populate the sql with subjects uploaded to Zooniverse
zoo.populate_subjects(zoo_info_dict["subjects"], project.value, db_path)

### Aggregrate video classifications based on appropriate agreement thresholds

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
# Select the workflow(s) to aggregrate the classifications
clips_class_df, workflow_names, workflows_df = t5.select_workflow(zoo_info_dict["classifications"], zoo_info_dict["workflows"], db_path)

In [ ]:
# Select the versions of the workflows to aggregrate the classifications
w_versions_list = t5.select_workflow_version(workflow_names.value, workflows_df)

In [ ]:
#Select only classifications from specific versions
clips_class_df["w_plus_version"] = clips_class_df['workflow_name']+"_"+clips_class_df['workflow_version']
clips_class_df = clips_class_df[clips_class_df.w_plus_version.isin(w_versions_list)]

# Specify the agreement threshold required among cit scientists
agg_params = t12.choose_agg_parameters("clip")

In [ ]:
# Aggregate the classifications
agg_class_df, raw_class_df = t12.aggregrate_classifications(clips_class_df, "clip", project.value, agg_params)

## Select the species of interest

In [ ]:
# Specify the species of interest
species_i = choose_species()

In [ ]:
# Filter the classifications to only species of interest


In [ ]:
# Check already uploaded frames for the species of interest (to avoid duplicates)

## List available frames to upload to Zooniverse

In [ ]:
# Specify the server to connect to
server_i = t_utils.choose_server()

In [ ]:
# Check availability of movies that correspond to the aggregated clips
t_utils.connect_server(server_i.value)

In [ ]:
#Specify how many frames per clip and how many clips of species i to upload


In [ ]:
# Set the subject_set to upload the frames to


### Preview the frames

In [ ]:
# Upload the frames to the project
